In [ ]:
import requests
import pandas as pd
import os, sys
from tqdm import tqdm

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from fpl_engineering.utils import get_project_root

pd.options.display.max_columns = None

: 

In [2]:
import gc
gc.collect()

0

In [25]:
sys.path

['/Users/bernie/Documents/Coding_stuff/Metis/DSML/Engineering/fpl_engineering/notebooks',
 '/Users/bernie/Documents/Coding_stuff/Metis/DSML/Engineering/fpl_engineering/notebooks',
 '/Users/bernie/miniforge3/envs/fpl_engineering/lib/python310.zip',
 '/Users/bernie/miniforge3/envs/fpl_engineering/lib/python3.10',
 '/Users/bernie/miniforge3/envs/fpl_engineering/lib/python3.10/lib-dynload',
 '',
 '/Users/bernie/.local/lib/python3.10/site-packages',
 '/Users/bernie/miniforge3/envs/fpl_engineering/lib/python3.10/site-packages',
 '/Users/bernie/Documents/Coding_stuff/Metis/DSML/Engineering/fpl_engineering',
 '/var/folders/m0/sh8zkxcs0sd0b_z8qd2zwrmm0000gn/T/tmpzw4dp3id']

In [28]:
url = "https://fantasy.premierleague.com/api/bootstrap-static/"

response = requests.get(url)
data = response.json()
data.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [7]:
elem_df = pd.DataFrame(data['elements'])
elem_df

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,penalties_order,penalties_text,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type
0,100.0,100.0,58822,0,0,-3,3,0,2,0.2,...,NaN,,540,160,350,118,362,134,397,146
1,NaN,NaN,84450,0,0,1,-1,1,3,4.5,...,NaN,,171,100,41,14,31,15,81,29
2,100.0,0.0,153256,0,0,-3,3,1,3,0.0,...,NaN,,547,282,478,176,261,104,227,71
3,NaN,NaN,156074,0,0,-3,3,0,2,0.2,...,NaN,,548,166,352,120,364,136,417,157
4,100.0,100.0,167199,0,0,-2,2,0,3,4.0,...,NaN,,266,163,62,21,59,27,249,83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,NaN,NaN,181284,-1,1,-1,1,0,3,0.6,...,NaN,,75,37,316,145,280,117,326,116
652,75.0,NaN,465351,0,0,-1,1,0,3,1.4,...,NaN,,247,149,209,92,200,73,357,127
653,0.0,0.0,429414,0,0,-2,2,0,4,0.0,...,NaN,,156,47,642,64,431,53,589,71
654,0.0,75.0,18507,0,0,0,0,0,4,0.0,...,NaN,,102,38,257,30,348,46,278,45


In [7]:
for row in elem_df.itertuples():
    print(row.Index)
    print(row['code'])
    break

0


TypeError: tuple indices must be integers or slices, not str

In [ ]:
# events: Basic information of every Gameweek such as average score, 
# highest score, top scoring player, most captained, etc.
data['events']

In [ ]:
# game_settings: The game settings and rules. (not important)
# phases: Phases of FPL season. (not really important)
data['phases']

In [ ]:
# teams: Basic information of current Premier League clubs.

data['teams']

In [ ]:
# elements: Information of all Premier League players including points, 
# status, value, match stats (goals, assists, etc.), ICT index, etc.

data['elements']


In [ ]:
data['element_stats']

In [27]:
fixtures_url = 'https://fantasy.premierleague.com/api/fixtures/'

f_response = requests.get(fixtures_url)
fixtures_data = f_response.json()

In [ ]:
fixtures_data, len(fixtures_data)

In [31]:
playes_url = f'https://fantasy.premierleague.com/api/element-summary/{4}/'

p_response = requests.get(playes_url)
playes_data = p_response.json()

In [35]:
playes_data.keys()

dict_keys(['fixtures', 'history', 'history_past'])

In [ ]:
len(playes_data['history']), playes_data['history']

In [77]:
def foo():
    return 'bar'

In [79]:
class MyClass(object):
    def __init__(self, download=True):
        self.download = download
        print("never called in this case")

    def __call__(self):
        if self.download:
            return 43
       

obj = MyClass()
obj()

never called in this case


43

In [26]:
class FPL():
    def __init__(self, data_dir='data', download=True) -> None:
        
        self.download = download
        self.data_dir = data_dir

        self.root = get_project_root()
        raw_path = os.path.join(self.data_dir, 'raw')
        raw_data_dir = os.path.join(self.root, raw_path)


        if self.download:
            print('Downloading latest Data from https://fantasy.premierleague.com. Hang tight!\n')
            all_his_list = []
            all_past_list = []
            url = "https://fantasy.premierleague.com/api/bootstrap-static/"
            r = requests.get(url)
            json = r.json()
            elem_df = pd.DataFrame(json['elements'])
            teams_df = pd.DataFrame(json['teams'])

            print(f'Fatching all statistics for all {len(elem_df)} players for current season\n')
            for row in tqdm(elem_df.itertuples(), total=len(elem_df), colour='green'):
                elem_id = row.id
                url = f'https://fantasy.premierleague.com/api/element-summary/{elem_id}/'
                r = requests.get(url)
                json = r.json()

                history_df = pd.DataFrame(json['history'])
                past_history = pd.DataFrame(json['history_past'])

                all_his_list.append(history_df)
                all_past_list.append(past_history)

            all_history_df = pd.concat(all_his_list)
            self.all_past_history_df = pd.concat(all_past_list)

            self.all_history_df = all_history_df.groupby('element', as_index=False, group_keys=False).apply(lambda x: x.iloc[:-1])
            self.all_history_df.reset_index().drop('index', axis=1)
            
            self.all_history_df.to_csv(str(raw_data_dir)+'/raw_history.csv', index=False)
            self.all_past_history_df.to_csv(str(raw_data_dir)+'/raw_past_history.csv', index=False)

        else:
            all_history_df = pd.read_csv(str(raw_data_dir)+'/raw_history.csv', )
            all_past_history_df = pd.read_csv(str(raw_data_dir)+'/raw_past_history.csv')

            self.all_history_df = all_history_df.groupby('element', as_index=False, group_keys=False).apply(lambda x: x.iloc[:-1])
            self.all_history_df.reset_index().drop('index', axis=1)
    
    def get_cleaned_df(self):
        processed_path = os.path.join(self.data_dir, 'processed')
        processed_data_dir = os.path.join(self.root, processed_path)

        self.cleaned_history = self.all_history_df.dropna(how = 'any')
        self.cleaned_history = self.cleaned_history.drop(['kickoff_time'], axis=1)
        self.cleaned_history[["influence", "creativity", "threat", "ict_index"]] = self.cleaned_history[["influence", "creativity", "threat", "ict_index"]].apply(pd.to_numeric)
        self.cleaned_history["was_home"] = self.cleaned_history["was_home"].astype(int)
       
        self.cleaned_history.to_csv(str(processed_data_dir)+'/cleaned_history.csv', index=False)
        return self.cleaned_history
        

In [4]:
# FPL()

In [27]:
db = FPL(download=False)
db.all_history_df

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out
0,1,1,7,0,False,2022-08-05T19:00:00Z,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,45,0,23970,0,0
1,1,11,10,0,True,2022-08-13T14:00:00Z,4,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,44,-5169,24193,1361,6530
2,1,21,3,0,False,2022-08-20T16:30:00Z,0,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,44,-4337,20960,879,5216
3,1,31,9,0,True,2022-08-27T16:30:00Z,2,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,43,-2988,18825,577,3565
4,1,41,2,0,True,2022-08-31T18:30:00Z,2,1,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,43,-1611,17790,405,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7813,645,117,11,0,True,2022-10-20T19:15:00Z,2,0,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,40,0,0,0,0
7814,645,130,20,0,False,2022-10-23T13:00:00Z,0,4,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,40,59,127,69,10
7816,646,121,2,0,False,2022-10-23T13:00:00Z,4,0,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,40,0,0,0,0
7818,647,127,1,0,True,2022-10-23T13:00:00Z,1,1,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,45,0,0,0,0


In [6]:
cleaned_all_history = db.get_cleaned_df()

In [9]:
len(cleaned_all_history.groupby('element'))

648

In [116]:
def make_sequences(df):    
    sequences = []
    val_sequences = []
    i = 0
    for series_id, group in df.groupby('element'):
        group.sort_values(by='round')
        s_len = 4
        d_size = len(group)
        # if player has played at least 5 games
        if d_size > s_len:
            for i in range(d_size-s_len-1):
                sequence = group[i:i+s_len]
                label = group.iloc[i+s_len]['total_points']
                sequences.append((sequence,label))
            val_sequences.append(sequences.pop())

    return sequences, val_sequences

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,value,transfers_balance,selected,transfers_in,transfers_out,total_points_y
0,1,1,7,0,False,2022-08-05T19:00:00Z,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,45,0,23970,0,0,0.0
1,1,11,10,0,True,2022-08-13T14:00:00Z,4,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,44,-5169,24193,1361,6530,0.0
2,1,21,3,0,False,2022-08-20T16:30:00Z,0,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,44,-4337,20960,879,5216,0.0
3,1,31,9,0,True,2022-08-27T16:30:00Z,2,1,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,43,-2988,18825,577,3565,0.0
4,1,41,2,0,True,2022-08-31T18:30:00Z,2,1,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,43,-1611,17790,405,2016,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,645,117,11,0,True,2022-10-20T19:15:00Z,2,0,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,40,0,0,0,0,0.0
1,645,130,20,0,False,2022-10-23T13:00:00Z,0,4,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,40,59,127,69,10,0.0
0,646,121,2,0,False,2022-10-23T13:00:00Z,4,0,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,40,0,0,0,0,0.0
0,647,127,1,0,True,2022-10-23T13:00:00Z,1,1,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,45,0,0,0,0,0.0


In [28]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from fpl_engineering.data.base_dataset import FPLDataset, make_sequences

# wraping the above into a pl datamodule
class FPLDataModule(pl.LightningDataModule):
    def __init__(self,batch_size=8, data_dir:str = 'data', download_data= True):
        super().__init__()
        self.batch_size = batch_size
        self.data_dir = data_dir
        self.download_data = download_data

    def prepare_data(self):
        # download
        data = FPL(self.data_dir, download=self.download_data)
        self.df = data.get_cleaned_df()
        self.train_sequences, self.val_sequences = make_sequences(self.df)

    def setup(self, stage:str):
        # Assign train/val datasets for use in dataloaders
        if stage == "fit":
            self.train_dataset = FPLDataset(self.train_sequences)
            self.val_dataset = FPLDataset(self.val_sequences)
    
        # Assign test dataset for use in dataloader(s)
        if stage == "test":
            self.test_dataset = FPLDataset(self.val_sequences)

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset, batch_size=self.batch_size,
            shuffle=False, num_workers=os.cpu_count()
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_dataset, batch_size=1,
            num_workers=4
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_dataset, batch_size=1,
            num_workers=4
        )


In [29]:
dm =FPLDataModule(download_data=False)
dm.prepare_data()
dm.setup(stage='fit')

<---------------------------------------------------------------->
making sequences from raw data



100%|██████████| 648/648 [00:00<00:00, 1905.74it/s]

Total train sequences made: 3321 
Total train Validation sequences made: 638 
Sequence length: 4


In [31]:
dm.train_sequences[3000][0].shape

(4, 35)

In [21]:
len(dm.train_sequences[100][0].columns)

NameError: name 'dm' is not defined

In [33]:
import onnx

ModuleNotFoundError: No module named 'onnx'

In [15]:
import torch
import torch.nn as nn


class FPLLSTMRegressor(pl.LightningModule):
    '''
    Standard PyTorch Lightning module:
    https://pytorch-lightning.readthedocs.io/en/latest/lightning_module.html
    '''
    def __init__(self, 
                 n_features, 
                 hidden_size, 
                 seq_len, 
                 batch_size,
                 num_layers, 
                 dropout, 
                 learning_rate):
        super(FPLLSTMRegressor, self).__init__()
        self.n_features = n_features
        self.hidden_size = hidden_size
        self.seq_len = seq_len
        self.batch_size = batch_size
        self.num_layers = num_layers
        self.dropout = dropout
        self.learning_rate = learning_rate

        self.criterion = nn.MSELoss()
        self.lstm = nn.LSTM(input_size=self.n_features, 
                                hidden_size=self.hidden_size,
                                num_layers=self.num_layers, 
                                dropout=self.dropout, 
                                batch_first=True)
        self.linear = nn.Linear(self.hidden_size, 1)

        self.save_hyperparameters()

        
    def forward(self, x):
        # lstm_out = (batch_size, seq_len, hidden_size)
        self.lstm.flatten_parameters()
        _, (hidden, _) = self.lstm(x)
        out = hidden[-1]
        y_pred = self.linear(out)
        return y_pred

    def common_step(self, batch):
        x = batch['sequence']
        y = batch['label']
        y_hat = self(x)
        loss = self.criterion(y_hat.squeeze(), y)
        return loss, y_hat

    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)

    def training_step(self, batch, batch_idx):
        loss, _ = self.common_step(batch)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, _ = self.common_step(batch)
        self.log('val_loss', loss)
        return loss

    
    def test_step(self, batch, batch_idx):
        loss, _ = self.common_step(batch)
        self.log('test_loss', loss)

## Train Loop / Function

In [16]:

'''
All parameters are aggregated in one place.
This is useful for reporting experiment params to experiment tracking software
'''

config = dict(
    seq_len = 4,
    batch_size = 16, 
    max_epochs = 10,
    n_features = 35,
    hidden_size = 128,
    num_layers = 1,
    dropout = 0.2,
    learning_rate = 0.001,
)

In [5]:
str(get_project_root())+'/data'

'/Users/bernie/Documents/Coding_stuff/Metis/DSML/Engineering/fpl_engineering/data'

In [19]:
import wandb
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import WandbLogger

from fpl_engineering.utils import get_project_root
project_root = str(get_project_root())

def train(config, run_name):
    # setup data
    #setup model
    with wandb.init(project='FPL', job_type='train', config=config) as run:
        config = wandb.config

        data_module = FPLDataModule(batch_size=config.batch_size, download_data=False)

        model = FPLLSTMRegressor(n_features= config.n_features, hidden_size=config.hidden_size, seq_len=config.seq_len, 
        batch_size=config.batch_size, num_layers=config.num_layers, dropout=config.dropout,learning_rate=config.learning_rate)

        # set up W&B logger
        checkpoint_callback = ModelCheckpoint(
        dirpath='logs/checkpoints',
        filename='best_checkpoint',
        save_top_k=1,
        verbose=True,
        monitor='val_loss',
        mode='min'
        )
        progress_bar = TQDMProgressBar(refresh_rate=30)

        
        
        logger = WandbLogger(name=run_name, save_dir='logs', project='FPL')
        progress_bar = TQDMProgressBar(refresh_rate=30)

        callbacks = [checkpoint_callback, progress_bar]

        # setup Trainer
        trainer = pl.Trainer(accelerator = 'cpu',
                            max_epochs = config.max_epochs,
                            logger= logger,
                            callbacks = callbacks,
                            log_every_n_steps=40,
                            precision=16)

        # train
        trainer.fit(model, datamodule=data_module)
        print(trainer.checkpoint_callback.best_model_path)

        model_artifact = wandb.Artifact(name='trained-model', type='model', 
                                        description='trained-lstm model',
                                        metadata=dict(config))

        best_model = FPLLSTMRegressor.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)
        torch.save(best_model,project_root+'/mpdels/lstm_model.pth')
        model_artifact.add_file('lstm_model.pth')

        best_model.to_onnx('lstm_model.onnx', export_params=True,
                input_names = ['input'],  
                output_names = ['output'], 
                dynamic_axes={'input' : {0 : 'batch_size'},'output' : {0 : 'batch_size'}})
                
        model_artifact.add_file('lstm_model.onnx')

        run.log_artifact(model_artifact)
        # wandb.save('trained_model.pth')
        
        wandb.finish()
        return trainer, data_module

In [20]:
train(config=config, run_name='run_001')

/Users/bernie/miniforge3/envs/fpl_engineering/lib/python3.10/site-packages/torch/nn/modules/rnn.py:67: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/Users/bernie/miniforge3/envs/fpl_engineering/lib/python3.10/site-packages/pytorch_lightning/loggers/wandb.py:387: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
/Users/bernie/miniforge3/envs/fpl_engineering/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:704: UserWarning: You passed `Trainer(accelerator='cpu', precision=16)` but native AMP is not supported on CPU. Using `precision='bf16'` instead.
  rank_zero_warn(
Using bfloat16 Automatic M

Epoch 0: 100%|██████████| 846/846 [7:48:17<00:00, 33.21s/it, loss=nan, v_num=jofs]

    self.run()
  File "/Users/bernie/miniforge3/envs/fpl_engineering/lib/python3.10/site-packages/wandb/sdk/service/server_sock.py", line 112, in run
    shandler(sreq)
  File "/Users/bernie/miniforge3/envs/fpl_engineering/lib/python3.10/site-packages/wandb/sdk/service/server_sock.py", line 173, in server_record_publish
    iface = self._mux.get_stream(stream_id).interface
  File "/Users/bernie/miniforge3/envs/fpl_engineering/lib/python3.10/site-packages/wandb/sdk/service/streams.py", line 199, in get_stream
    stream = self._streams[stream_id]
KeyError: '1bo1jofs'



<---------------------------------------------------------------->
making sequences from raw data



100%|██████████| 648/648 [00:00<00:00, 1603.05it/s]

  | Name      | Type    | Params
--------------------------------------
0 | criterion | MSELoss | 0     
1 | lstm      | LSTM    | 84.5 K
2 | linear    | Linear  | 129   
--------------------------------------
84.6 K    Trainable params
0         Non-trainable params
84.6 K    Total params
0.338     Total estimated model params size (MB)


...

Epoch 9, global step 2080: 'val_loss' was not in top 1
`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 846/846 [01:11<00:00, 11.88it/s, loss=2.57, v_num=y96b]
/Users/bernie/Documents/Coding_stuff/Metis/DSML/Engineering/fpl_engineering/notebooks/logs/checkpoints/best_checkpoint.ckpt


/Users/bernie/miniforge3/envs/fpl_engineering/lib/python3.10/site-packages/torch/nn/modules/rnn.py:67: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


wandb: ERROR Control-C detected -- Run data was not synced


RuntimeError: Parent directory /Users/bernie/Documents/Coding_stuff/Metis/DSML/Engineering/fpl_engineering/mpdels does not exist.

In [ ]:
wandb.finish()